In [21]:

import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
from matplotlib import pyplot as plt
np.random.seed(42)


class Scaler():
#     mean 
#     std 
    # hint: https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/
    def __call__(self,features, is_train=False):
#         if(is_train):
#             Scaler.mean = np.mean(features,axis=0)
#             Scaler.std = np.std(features,axis=0)
#         print(features.shape)
#         features[:,0:] = (features[:,0:] - Scaler.mean)/ Scaler.std
        min_f = np.min(features[:,0:])
        max_f = np.max(features[:,0:])
        features[:,0:] = (features[:,0:]-min_f)/(max_f-min_f)
        return features


def get_features(csv_path,is_train=False,scaler=None):
    '''
    Description:
    read input feature columns from csv file
    manipulate feature columns, create basis functions, do feature scaling etc.
    return a feature matrix (numpy array) of shape m x n 
    m is number of examples, n is number of features
    return value: numpy array
    '''
    '''
    Arguments:
    csv_path: path to csv file
    is_train: True if using training data (optional)
    scaler: a class object for doing feature scaling (optional)
    '''

    '''
    help:
    useful links: 
        * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
        * https://www.geeksforgeeks.org/python-read-csv-using-pandas-read_csv/
    '''
    df = pd.read_csv(csv_path)
    features = np.array(df.drop(' shares',axis = 1))
    '''Scaling'''
    scaler.__call__(features,is_train)
    '''Bias column addtition'''
    bias = np.ones((1,len(features[:,0])))
    features = np.insert(features, 0, bias, axis=1)
    return features
    
    

def get_targets(csv_path):
    '''
    Description:
    read target outputs from the csv file
    return a numpy array of shape m x 1
    m is number of examples
    '''
    df = pd.read_csv(csv_path)
    target = df[' shares'].values.reshape(-1,1)
    Tnorm = target.copy()
    max_t = np.max(target)
    min_t = np.min(target)
    Tnorm = (target-min_t)/(max_t - min_t)
    return Tnorm
     

def analytical_solution(feature_matrix, targets, C=0.0):
    '''
    Description:
    implement analytical solution to obtain weights
    as described in lecture 5d
    return value: numpy array
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    targets: numpy array of shape m x 1
    '''
    p1 = np.dot(feature_matrix.T,feature_matrix)
    p2 = p1 + C*np.identity(len(feature_matrix[0]))
    p3 = np.dot(feature_matrix.T,targets)
    w_star = np.matmul(np.linalg.pinv(p2),p3)
    return w_star

def get_predictions(feature_matrix, weights):
    '''
    description
    return predictions given feature matrix and weights
    return value: numpy array
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    weights: numpy array of shape n x 1
    '''

    pre = feature_matrix.dot(weights)
    return pre

def mse_loss(feature_matrix, weights, targets):
    '''
    Description:
    Implement mean squared error loss function
    return value: float (scalar)
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    weights: numpy array of shape n x 1
    targets: numpy array of shape m x 1
    '''
    loss = np.sum((feature_matrix.dot(weights)-targets)**2,axis=0)[0]
    return loss  

def l2_regularizer(weights):
    '''
    Description:
    Implement l2 regularizer
    return value: float (scalar)
    '''

    '''
    Arguments
    weights: numpy array of shape n x 1
    '''
    raise NotImplementedError

def loss_fn(feature_matrix, weights, targets, C=0.0):
    '''
    Description:
    compute the loss function: mse_loss + C * l2_regularizer
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    weights: numpy array of shape n x 1
    targets: numpy array of shape m x 1
    C: weight for regularization penalty
    return value: float (scalar)
    '''

    raise NotImplementedError

def compute_gradients(feature_matrix, weights, targets, C=0.0):
    '''
    Description:
    compute gradient of weights w.r.t. the loss_fn function implemented above
    '''
    
    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    weights: numpy array of shape n x 1
    targets: numpy array of shape m x 1
    C: weight for regularization penalty
    return value: numpy array
    '''
    m = len(targets)
#     grad = 1/m*np.sum((np.dot(feature_matrix,weights) - targets)*feature_matrix,axis=0).reshape(-1,1)

    grad = (2/m)*np.dot(feature_matrix.T,(np.dot(feature_matrix,weights) - targets))
    
#     grad = grad + 2*C*weights
    return grad
    
#     loss = np.subtract(np.dot(feature_matrix,weights),targets)
#     gradient_w = np.dot(feature_matrix.T,loss) 
#     return gradient_w

def sample_random_batch(feature_matrix, targets, batch_size):
    '''
    Description
    Batching -- Randomly sample batch_size number of elements from feature_matrix and targets
    return a tuple: (sampled_feature_matrix, sampled_targets)
    sampled_feature_matrix: numpy array of shape batch_size x n
    sampled_targets: numpy array of shape batch_size x 1
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    targets: numpy array of shape m x 1
    batch_size: int
    '''    
    lb = np.random.randint(0, len(feature_matrix) - batch_size)
    ub = lb + batch_size
    return (feature_matrix[lb:ub], targets[lb:ub])
    
    
def initialize_weights(n):
    '''
    Description:
    initialize weights to some initial values
    return value: numpy array of shape n x 1
    '''

    '''
    Arguments
    n: int
    '''
    weights = np.random.randn(n,1)
    return weights
    
def update_weights(weights, gradients, lr):
    '''
    Description:
    update weights using gradient descent
    retuen value: numpy matrix of shape nx1
    '''

    '''
    Arguments:
    # weights: numpy matrix of shape nx1
    # gradients: numpy matrix of shape nx1
    # lr: learning rate
    '''    

    weights = weights - lr*gradients
    return weights

def early_stopping(arg_1=None, arg_2=None, arg_3=None, arg_n=None):
    # allowed to modify argument list as per your need
    # return True or False
    raise NotImplementedError
    

def do_gradient_descent(train_feature_matrix,  
                        train_targets, 
                        dev_feature_matrix,
                        dev_targets,
                        lr=1.0,
                        C=0.0,
                        batch_size=32,
                        max_steps=10000,
                        eval_steps=5):
    '''
    feel free to significantly modify the body of this function as per your needs.
    ** However **, you ought to make use of compute_gradients and update_weights function defined above
    return your best possible estimate of LR weights
    a sample code is as follows -- 
    '''
    weights = initialize_weights(len(train_feature_matrix[0]))
    dev_loss = mse_loss(dev_feature_matrix, weights, dev_targets)
    train_loss = mse_loss(train_feature_matrix, weights, train_targets)

    print("step {} \t dev loss: {} \t train loss: {}".format(0,dev_loss,train_loss))
    for step in range(1,max_steps+1):

        #sample a batch of features and gradients
        features,targets = sample_random_batch(train_feature_matrix,train_targets,batch_size)
        
        #compute gradients
        gradients = compute_gradients(features, weights, targets, C)
        
        #update weights
        weights = update_weights(weights, gradients, lr)

        if step%eval_steps == 0:
            dev_loss = mse_loss(dev_feature_matrix, weights, dev_targets)
            train_loss = mse_loss(train_feature_matrix, weights, train_targets)
            print("step {} \t dev loss: {} \t train loss: {}".format(step,dev_loss,train_loss))

        '''
        implement early stopping etc. to improve performance.
        '''

    return weights

def do_evaluation(feature_matrix, targets, weights):
    # your predictions will be evaluated based on mean squared error 
    predictions = get_predictions(feature_matrix, weights)
    loss =  mse_loss(feature_matrix, weights, targets)
    return loss

if __name__ == '__main__':
    scaler = Scaler() #use of scaler is optional
    train_features, train_targets = get_features('train.csv',True,scaler), get_targets('train.csv')
    dev_features, dev_targets = get_features('dev.csv',False,scaler), get_targets('dev.csv')

    a_solution = analytical_solution(train_features, train_targets, C=1e-8)
    print('evaluating analytical_solution...')
    dev_loss=do_evaluation(dev_features, dev_targets, a_solution)
    train_loss=do_evaluation(train_features, train_targets, a_solution)
    print('analytical_solution \t train loss: {:.10f}, dev_loss: {:.10f} '.format(train_loss, dev_loss))

    print('training LR using gradient descent...')
    gradient_descent_soln = do_gradient_descent(train_features, 
                        train_targets, 
                        dev_features,
                        dev_targets,
                        lr=0.1,
                        C=1e-15,
                        batch_size=32,
                        max_steps=50000,
                        eval_steps=10000)

    print('evaluating iterative_solution...')
    dev_loss=do_evaluation(dev_features, dev_targets, gradient_descent_soln)
    train_loss=do_evaluation(train_features, train_targets, gradient_descent_soln)
    print('gradient_descent_soln \t train loss: {}, dev_loss: {} '.format(train_loss, dev_loss))

evaluating analytical_solution...
analytical_solution 	 train loss: 0.0000072296, dev_loss: 8.0581447619 
training LR using gradient descent...
step 0 	 dev loss: 7252.073406710706 	 train loss: 40297.424982539334
step 10000 	 dev loss: 9.779444384830512 	 train loss: 5.076865221629813
step 20000 	 dev loss: 8.454180350132562 	 train loss: 2.338349732630941
step 30000 	 dev loss: 8.842297889736564 	 train loss: 1.4526437607665141
step 40000 	 dev loss: 8.441756060905334 	 train loss: 1.1216760097796064
step 50000 	 dev loss: 8.475664354466979 	 train loss: 0.9674275230444365
evaluating iterative_solution...
gradient_descent_soln 	 train loss: 0.9674275230444365, dev_loss: 8.475664354466979 


In [190]:
df = pd.read_csv('test.csv')
fx = df.to_numpy()
min_fx = np.min(fx[:,0:])
max_fx = np.max(fx[:,0:])
fx[:,0:] = (fx[:,0:]-min_fx)/(max_fx-min_fx)
# Adding bias
bias_x = np.ones((1,len(fx[:,0])))
fx = np.insert(fx, 0, bias_x, axis=1)
print(fx.shape)
predictions = get_predictions(fx, a_solution)
print(predictions[1:3])
# Denormalize
df = pd.read_csv('train.csv')
target = df[' shares'].values.reshape(-1,1)
max_t = np.max(target)
min_t = np.min(target)
predictions = (predictions*(max_t - min_t))+min_t
print(predictions[1:3])
csv = pd.DataFrame()
csv['instance_id'] = [i for i in range(len(predictions))]

(11894, 60)
[[0.24159223]
 [0.17863273]]
[[5504.66023413]
 [4463.12131597]]


In [191]:
csv['shares'] = predictions
print(csv)
csv.to_csv('submission.csv',index=False)

       instance_id       shares
0                0  5995.413504
1                1  5504.660234
2                2  4463.121316
3                3  5568.774562
4                4  5962.962725
...            ...          ...
11889        11889  7017.171565
11890        11890  5574.931788
11891        11891  6187.125392
11892        11892  1918.112203
11893        11893  4729.084585

[11894 rows x 2 columns]


In [1]:
import pandas as pd
import numpy as np
import matplotlib
import tkinter
matplotlib.use("TkAgg")
from matplotlib import pyplot as plt
np.set_printoptions(suppress=True)

In [5]:
train = pd.read_csv('train.csv')
features = train.values
target = train[' shares'].values.reshape(-1,1)
features = np.delete(features, 59, 1)
# target = np.delete(target, np.s_[99:23786:1], 0)
# features = np.delete(features, np.s_[99:23786:1], 0)
print("targets shape:",target.shape,"\tfeatures shape:",features.shape)
# print("vector=",features[:,2])
# print("mean=",np.mean(features[:,2]),"std=",np.std(features[:,2]))

# trial = (features[:,2] - np.mean(features[:,2])) / np.std(features[:,2])
# mean = np.mean(features,axis=0)
# std = np.std(features,axis=0)
# features[:,0:] = (features[:,0:] - mean)/ std
min_f = np.min(features[:,0:])
max_f = np.max(features[:,0:])
features[:,0:] = (features[:,0:]-min_f)/(max_f-min_f)
# print(np.mean(features,axis=0))
# print(np.std(features,axis=0))
# max_t = np.max(features[:,2])
# min_t = np.min(features[:,2])
# print("mean=",np.mean(features,axis=0),"std=",np.std(features[:,2]))
# features[:,2] = (features[:,2]-min_t)/(max_t - min_t)

# # print("vector=",features[:,2])
# print("mean=",np.mean(features[:,2]),"std=",np.std(features[:,2]))
# print(train.head())
# list = []
# for i in range(0,23786):
#     list.append(i)
# X = np.array(list).reshape(-1,1)
# print(X.shape)
# fig = plt.figure(1, figsize=(12,12))
# ax = fig.add_subplot(121)
# ax1 = fig.add_subplot(122)
# ax.scatter(X,features[:,2],c='g')
# ax1.scatter(X,features[:,2],c='r')
# plt.show()
bias = np.ones((1,len(features[:,0])))
features = np.insert(features,0,bias,axis=1)
# print(X[0:3,0:3])
# print(np.std(X,axis=0))
# print(target[1:4])
# Tnorm = target.copy()
max_t = np.max(target)
min_t = np.min(target)
target = (target-min_t)/(max_t - min_t)
# max_t1 = np.max(Tnorm)
# min_t1 = np.min(Tnorm)
# print(min_t,max_t)
# print(min_t1,max_t1)
# weights = np.random.randn(len(features[0]),1)
# print (0*weights)
print("targets shape:",target.shape,"\tfeatures shape:",features.shape)

targets shape: (23786, 1) 	features shape: (23786, 59)
targets shape: (23786, 1) 	features shape: (23786, 60)


In [20]:
weights = np.random.randn(len(features[0,:]),1)
m = len(target)
p1 = features.dot(weights) - target
p2 = np.dot(features.T,p1)
# p3 = np.sum(p2,axis=0)
# p1 = (feature_matrix.dot(weights) - targets)*feature_matrix
# p2 = p1 + C*weights
# grad = 1/m*np.sum((,axis=0) + C*weights).reshape(-1,1)
# grad = 1/m*np.sum(p2,axis=0).reshape(-1,1)


    
# loss = np.subtract(np.dot(feature_matrix,weights),targets)
# gradient_w = np.dot(feature_matrix.T,loss) 
# return gradient_w
print(weights.shape,"\tX.W - Y ==",p1.shape,"\t(X.W-Y)*X.T ==",p2.shape)
# print("sum() ==",p3.shape)

(60, 1) 	X.W - Y == (23786, 1) 	(X.W-Y)*X.T == (60, 1)


In [5]:
X = np.concatenate((np.ones((len(features),1)),features),axis=1)
Xnorm = X.copy()
minx = np.min(X[:,1:])
maxx = np.max(X[:,1:])
Xnorm[:,1:] = (X[:,1:]-minx)/(maxx-minx)

ynorm = target.copy()
maxy = np.max(target)
miny = np.min(target)
ynorm = (target-miny)/(maxy - miny) 
print("xnorm=",Xnorm.shape,"ynorm=",ynorm.shape)

theta0 = np.zeros((X.shape[1],1))+0.4
print("theta=",theta0.shape)

ypred = Xnorm.dot(theta0)
print("ypred=",ypred.shape)
sortidx = np.argsort(ynorm[:,0]) # sort the values for better visualization
plt.plot(ynorm[sortidx,0],'o')
plt.plot(ypred[sortidx,0],'--')
plt.show()

xnorm= (23786, 60) ynorm= (23786, 1)
theta= (60, 1)
ypred= (23786, 1)


In [6]:
# calculate gradient
def grad(theta):
    dJ = 1/m*np.sum((Xnorm.dot(theta)-ynorm)*Xnorm,axis=0).reshape(-1,1)
    return dJ


In [7]:
def cost(theta):
    J = np.sum((Xnorm.dot(theta)-ynorm)**2,axis=0)[0]
    return J

cost(theta0)

10360.122353200131

In [8]:
def GD(theta0,learning_rate = 0.5,epochs=50000,TOL=1e-7):
    
    theta_history = [theta0]
    cost_history = [cost(theta0)]
    
    theta_star = theta0*10000
    print(f'epoch \t Cost(J) \t')
    for epoch in range(epochs):
        if epoch%100 == 0:
            print(f'{epoch:5d}\t{cost_history[-1]:7.4f}\t')
        dJ = grad(theta0)
        J = cost(theta0)
        
        theta_star = theta0 - learning_rate*dJ
        theta_history.append(theta_star)
        cost_history.append(J)
        
        if np.sum((theta_star - theta0)**2) < TOL:
            print('Convergence achieved.')
            break
        theta0 = theta_star

    return theta_star,theta_history,cost_history

In [9]:
theta,theta_history,cost_history = GD(theta0)

epoch 	 Cost(J) 	
    0	10360.1224	
  100	13.9842	
  200	 8.5113	
  300	 5.7978	
  400	 4.1081	
  500	 2.9595	
Convergence achieved.


In [10]:
plt.plot(cost_history)
plt.show()

In [11]:
yprednorm = Xnorm.dot(theta)

ypred = yprednorm*(maxy-miny) + miny
plt.plot(target[sortidx,0],'o')
plt.plot(ypred[sortidx,0],'--')
plt.show()

In [41]:
# x = np.array(x)
# y = np.array(y).reshape((100,1))
# loss = np.subtract(np.dot(x,np.random.randn(len(x[0]),1)),y)
df = pd.read_csv('train.csv')
targets = df[' shares'].values.reshape(-1,1)

feature_matrix = Xnorm.copy()

# gradient_w = -2*np.dot(x.T,loss)
# gradient_w * 2 * 0.1
p1 = np.dot(feature_matrix.T,feature_matrix)
p2 = p1 + 0.4*np.identity(len(feature_matrix[0]))
p3 = np.dot(feature_matrix.T,targets)
w_star = np.matmul(np.linalg.pinv(p2),p3)

ypred=np.dot(feature_matrix,w_star)

plt.plot(targets[sortidx,0],'o')
plt.plot(ypred[sortidx,0],color='red')
plt.title("Anal Solution")
plt.xlabel("Instances")
plt.ylabel("Targets")
plt.legend()
plt.show()

No handles with labels found to put in legend.


In [6]:
e.shape

(60, 60)